In [295]:
import os
import selenium
from selenium import webdriver
import time
from PIL import Image
import io
import requests
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
import datetime as dt
import pandas as pd
import csv
import collections

In [331]:
# Install and initiate driver
driver = webdriver.Chrome(ChromeDriverManager().install())

# URL from brand pages
# adidas_url = 'https://stockx.com/adidas'
# jordan_url = 'https://stockx.com/retro-jordans'
# nike_url = 'https://stockx.com/nike'
# other_sneakers_url = 'https://stockx.com/other-sneakers'
luxury_sneakers_url = 'https://stockx.com/luxury-brands'

# open and maximize the browser window
# (change url if needed)
driver.get(luxury_sneakers_url)
driver.maximize_window()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430
[WDM] - Driver [/Users/gabbyvinco/.wdm/drivers/chromedriver/mac64/90.0.4430.24/chromedriver] found in cache


In [332]:
# initiate the action object
action = webdriver.ActionChains(driver)

# get rid of the pop up message upon opening the website
wait = WebDriverWait(driver, 10)

pop_up_confirm = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[6]/div[4]/div/section/footer/button')))

pop_up_confirm.click()

In [333]:
all_xpaths = []
all_hrefs = []
shoe_link = []

In [334]:
# create xpath routes for all the links on the page
for tile in range (1,21):
    # no need to adjust tile path with the change of url
    tile_path = "/html/body/div[1]/div[1]/div[2]/div[2]/div[3]/div/div/div[2]/div[2]/div[2]/div[2]/div["+str(tile)+"]/div/a"
    all_xpaths.append(tile_path)
#print(all_xpaths)


In [335]:
# scrape all the href links for each tile on the page
def scrape_href():
    for path in all_xpaths:
        link = driver.find_element_by_xpath(path).get_attribute('href')
        all_hrefs.append(link)
    #print(all_hrefs)

In [336]:
# split to isolate the link needed for the request
def split_href():
    for href in all_hrefs:
        link = href.split("m/")
        shoe_link.append(link[1])
    #print("links added to list")

In [337]:
# automate it to click through all the pages until we have a duplicate link
page_count = 0
tile_count = 0
# make sure lists are empty before scraping
all_hrefs = []
shoe_link = []

# range 1 to 51 because there are a maximum number of 50 pages to go through
while page_count <= 30:
    page_count += 1 # add to count so you know how many iterations you're doing
    tile_count += 20 # add to tile count so you can check to see if the current page duplicates with the first page
    scrape_href() # get hrefs
    split_href() # split hrefs
    driver.refresh()
    # next button is also the same regardless of switch in url
    element = driver.find_element_by_xpath("/html/body/div[1]/div[1]/div[2]/div[2]/div[3]/div/div/div[2]/div[3]/ul/a[7]")
    driver.execute_script("arguments[0].click();", element) 
    # skip the first iteration becuase we know there aren't any duplicates on the first page
    if page_count == 1:
        print("working....")
        continue
    # if not first iteration, then we need to check for duplicates
    elif page_count != 1:
        first_shoe = shoe_link[0]
        counter=collections.Counter(shoe_link)
        unique_shoes = list(counter.keys())
        if unique_shoes.count(first_shoe) > 1:
            print("duplicate found")
            break
        else:
            print("working")
            continue
        


working....
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working


In [338]:
print(unique_shoes)

['louis-vuitton-don-kanye-red', 'alexander-mcqueen-oversized-ivory-black', 'gucci-rubber-slides-red-green', 'off-white-low-vulc-white-black-aw20', 'off-white-industrial-slides-black-fw20', 'gucci-bambi-web-silver', 'lv-trainer-white-ss21', 'gucci-web-slide-sandal-black', 'gucci-pursuit-pool-slides-white', 'gucci-slides-bloom-supreme', 'balenciaga-speed-trainers-clearsole', 'balenciaga-track-black', 'gucci-flashtrek-sega-green-removable-crystal', 'balenciaga-speed-20-black', 'off-white-odsy-1000-light-blue-fw20', 'balenciaga-speed-20-black-white', 'dior-b23-high-top-logo-oblique', 'alexander-mcqueen-oversized-ivory-black-w', 'balenciaga-triple-s-black-2019', 'common-projects-original-achilles-white', 'alexander-mcqueen-oversized-worker-blue', 'balenciaga-triple-s-neon-green', 'alexander-mcqueen-oversized-black', 'dior-b22-white-red', 'balenciaga-triple-s-allover-logo-white', 'alexander-mcqueen-oversized-shock-pink', 'off-white-vulc-low-white-red-arrow', 'off-white-slides-industrial', 'o

In [339]:
len(unique_shoes)

494

In [340]:
counter=collections.Counter(shoe_link)
# adidas_unique_shoes = list(counter.keys())
# jordan_unique_shoes = list(counter.keys())
# nike_unique_shoes = list(counter.keys())
# other_unique_shoes = list(counter.keys())
luxury_unique_shoes = list(counter.keys())

In [341]:
# save as csv to loop through later in requests
# (make sure to change the name you are saving the csv to)
with open('luxury_links.csv', 'w', newline='\n') as csvfile:
    csvwriter = csv.writer(csvfile)
#     csvwriter.writerow(adidas_unique_shoes)
#     csvwriter = csvwriter.writerow(jordan_unique_shoes)
#     csvwriter = csvwriter.writerow(nike_unique_shoes)
#     csvwriter = csvwriter.writerow(other_unique_shoes)
    csvwriter = csvwriter.writerow(luxury_unique_shoes)